In [29]:
import cnvpytor
import subprocess
print(cnvpytor.__version__)

1.3.1


In [30]:
filename = "test_with_del_400_1_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

1129759

In [31]:
read_count = line_count * 50 *30/202
read_count = round(read_count)
read_count

8389300

In [32]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700245518
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 56487872


b'chr22\t1804\tC\tA\t-\nchr22\t3483\tG\tS\t+\nchr22\t4358\tT\tA\t-\nchr22\t4658\tT\tY\t+\nchr22\t6631\tT\tY\t+\nchr22\t7785\tA\tM\t+\nchr22\t8609\tC\t-\t-\nchr22\t9853\tC\tM\t+\nchr22\t9899\tA\tW\t+\nchr22\t12637\tT\tY\t+\nchr22\t14912\tT\tW\t+\nchr22\t15906\t-\tA\t+\nchr22\t15950\tT\tK\t+\nchr22\t16450\tTC\t-\t-\nchr22\t16468\tC\tM\t+\nchr22\t17024\tA\tM\t+\nchr22\t17249\tG\tK\t+\nchr22\t18095\tA\tM\t+\nchr22\t18872\tC\t-\t-\nchr22\t21457\tA\tM\t+\nchr22\t21852\tT\tW\t+\nchr22\t23098\tA\tR\t+\nchr22\t23361\tC\tG\t-\nchr22\t24320\tG\tR\t+\nchr22\t25653\tG\tK\t+\nchr22\t26927\tC\tA\t-\nchr22\t27031\tG\tR\t+\nchr22\t27114\tA\tM\t+\nchr22\t27444\tA\tC\t-\nchr22\t28041\tT\tY\t+\nchr22\t29893\tG\tA\t-\nchr22\t30199\tA\tW\t+\nchr22\t32463\tT\tY\t+\nchr22\t33398\tC\tA\t-\nchr22\t33838\tC\tG\t-\nchr22\t33911\t-\tG\t+\nchr22\t45832\tT\tK\t+\nchr22\t47320\tG\tC\t-\nchr22\t47623\tG\tS\t+\nchr22\t47725\tC\tM\t+\nchr22\t48883\tT\tY\t+\nchr22\t50779\tT\tY\t+\nchr22\t50983\tG\tK\t+\nchr22\t52107\t-\t

In [33]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [34]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 10 files and 1 in-memory blocks...


In [35]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [36]:
app.rd(["biasOut_sorted.bam"])

In [37]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [38]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [39]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.9921	1.644525e-06	2.432068e-86	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047900	8500	2.9776	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13480601-13513300	32700	7.9394	0.000000e+00	1.889965e-23	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:18662501-18667700	5200	1.9710	0.000000e+00	1.567216e-20	1.308080e-05	1.113484e-11	0.0004	0.0000	1814800
duplication	chr22:18680401-18682700	2300	1.9953	4.273476e-03	2.056052e+00	1.295189e+05	2.444030e-11	0.0000	0.0000	1826900
duplication	chr22:18704101-18881400	177300	5.3182	0.000000e+00	2.275424e-57	0.000000e+00	0.000000e+00	0.0002	0.0000	1628200
duplication	chr22:20335901-20338300	2400	1.9135	2.174601e-03	4.638640e-08	2.093272e+05	4.564846e+06	0.0000	0.0000	171300
duplication	chr22:20338601-20340900	2300	1.8370	3.634676e-02	2.742169e-19	3.193378e+05	5.183709e-08	0.0019	0.0000	168700
duplication	chr22:20341301-20344500	3200	2.1775

In [40]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [52]:
print(f"{filename=}, Histogram Size: 100")
print('type,chromosome_name,start,end,size,cnv_level,q0')

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

filename='test_with_del_400_1_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.9921069923257135,0.0
duplication,chr22,8039401,8047900,8500,2.9776482923868635,0.0
duplication,chr22,13480601,13513300,32700,7.939444863277274,0.0
duplication,chr22,18662501,18667700,5200,1.9709579056744801,0.00038231171148209506
duplication,chr22,18680401,18682700,2300,1.9953447993487736,0.0
duplication,chr22,18704101,18881400,177300,5.318181948739732,0.00015384092986447446
duplication,chr22,20335901,20338300,2400,1.9135073048328872,0.0
duplication,chr22,20338601,20340900,2300,1.8369545378599912,0.0018547140649149924
duplication,chr22,20341301,20344500,3200,2.1774696465211725,0.0013123359580052493
duplication,chr22,20345701,20347800,2100,1.778367074792166,0.0
duplication,chr22,20349201,20357000,7800,1.552210445331492,0.0005932477618379895
duplication,chr22,20357501,20361100,3600,1.7835316535243886,0.0
duplication,chr22,20363901,2036630

In [49]:
print(f"{filename=}, Histogram Size: 1000")
print('type,chromosome_name,start,end,size,cnv_level,q0')

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

filename='test_with_del_400_1_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,8040001,8048000,8000,2.9704977216326194,0.0
duplication,chr22,13481001,13513000,32000,8.049257412365229,0.0
duplication,chr22,18658001,18683000,25000,1.60915345766002,0.00041953754054184885
duplication,chr22,18705001,18881000,176000,5.320488693074513,0.00011267605633802817
duplication,chr22,20326001,20368000,42000,1.6548820347955073,0.00046664862291991374
duplication,chr22,20610001,20655000,45000,2.080531788919076,0.0001596490497410909
duplication,chr22,21466001,21505000,39000,1.7381756742881092,0.00019154336062826222
duplication,chr22,21522001,21559000,37000,1.6756902740988295,0.0018435303606406269
duplication,chr22,21561001,21686000,125000,1.9209141244430774,0.0005301137039801803
duplication,chr22,23608001,23613000,5000,3.976525960397803,0.0
duplication,chr22,28466001,28599000,133000,5.963115601600853,4.425141358682291e-05
duplication,chr22,30804001,3098

In [50]:
print(f"{filename=}, Histogram Size: 10,000")
print('type,chromosome_name,start,end,size,cnv_level,q0')

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

filename='test_with_del_400_1_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,18650001,18880000,230000,4.284945521899749,0.00016134394299874633
duplication,chr22,21460001,21680000,220000,1.7667120686116262,0.00046880376904930916
duplication,chr22,28460001,28600000,140000,5.697184550572088,4.378919234078371e-05
duplication,chr22,30800001,30980000,180000,5.897815443443115,1.4724061877870042e-05
duplication,chr22,35930001,36200000,270000,3.8761359272552243,2.9714425796578755e-05
duplication,chr22,44150001,44460000,310000,3.9987432548659134,2.5141288805317384e-05
duplication,chr22,46380001,46670000,290000,3.001614813715323,2.092838307312377e-05


In [51]:
print(f"{filename=}, Histogram Size: 100,000")
print('type,chromosome_name,start,end,size,cnv_level,q0')

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

filename='test_with_del_400_1_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0


In [45]:
print(f"{filename=}, Histogram Size: 1,000,000")
print('type,chromosome_name,start,end,size,cnv_level,q0')

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 1,000,000


In [46]:
print(f"{filename=}, Histogram Size: 10,000,000")
print('type,chromosome_name,start,end,size,cnv_level,q0')

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 10,000,000


In [47]:
del app